This notebook has the purpose of comparing tidal ellipses calculated by the model to other data sources.

#Loading and Preparing the Files

The following cells in this section will load, mask, rotate and unstagger the model output data. 

In [56]:
import os
import glob
import datetime
import matplotlib.pylab as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex
import datetime
import pandas as pd

import netCDF4 as nc
from scipy.optimize import curve_fit
from salishsea_tools import (viz_tools,tidetools, nc_tools)
from salishsea_tools.nowcast import (research_VENUS, analyze)

%matplotlib inline

In [57]:
grid_B = nc.Dataset('/data/dlatorne/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc')
lats = grid_B.variables['nav_lat'][:]
lons = grid_B.variables['nav_lon'][:]
bathy = grid_B.variables['Bathymetry'][:,:]

In [58]:
to = datetime.datetime(2014,11,24)
tf = datetime.datetime(2015, 6, 8)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

runname1 = '{}_{}_{}_Central'.format(t_o, t_f, 'h')
runname2 = '{}_{}_{}_East'.format(t_o, t_f, 'h')

#Only importing M2 parameters
cols=np.arange(0,4)

df1 = pd.read_csv(runname1+'.csv', usecols=cols) 
df2 = pd.read_csv(runname2+'.csv', usecols=cols)

cen = df1.as_matrix()
east = df2.as_matrix()

The locations of the nodes are stored in a dictionary in research_VENUS.py

Slice in order to compare, the min and max depth values for each location is determine by the comparison source.

In [59]:
central_min = 35
central_max = 290

east_min = 20
east_max = 160

minc = [n for n,i in enumerate(cen[:,0]) if i>central_min ][0]
maxc = [n for n,i in enumerate(cen[:,0]) if i>central_max ][0]
mine = [n for n,i in enumerate(east[:,0]) if i>east_min ][0]
maxe = [n for n,i in enumerate(east[:,0]) if i>east_max ][0]        

cen_slice = cen[minc:maxc]
east_slice = east[mine:maxe]

In [60]:
cen_av = analyze.depth_average(cen_slice,cen_slice[:,0],0)

east_av = analyze.depth_average(east_slice,east_slice[:,0],0)

#Fits

The following cells will calculate the cell-centered model output velocities' tidal parameters.

#Comparisons
##Depth Averaged at VENUS nodes 

Mark Halverson's values:

In [61]:
majc = 0.179
minorc = 0.05
incc = 109

#CODAR values
majc_cod = 0.113
minorc_cod = 0.08
incc_cod = 127

maje = 0.262
minore = -0.04
ince = 124

maje_cod = 0.138
minore_cod = -0.037
ince_cod = 133

The nodes were depth averaged between 35 - 290 m at the Central node and 20 - 160 m at the East node. I have done the same with the model output. Note that the node values wer averaged about 10-11 months where are the model output values were averages over 6 1/2 months (Nov 24, 2014 to June 8, 2015).

In [62]:
print "Central   Major-Axis   Minor-Axis     Angle"
print "           (m/s)      (m/s)     (deg ccw E)"  
print ' Model      {0:.2f}       {1:.2f}      {2:.0f}   '.format( 
         cen_av[1], cen_av[2], cen_av[3]) 
print ' Nodes      {0:.2f}       {1:.2f}      {2:.0f}   '.format(
         majc, minorc, incc)
#print ' CODAR      {0:.2f}       {1:.2f}      {2:.0f}   '.format(
#         majc_cod, minorc_cod, incc_cod)


print "East   Major-Axis   Minor-Axis     Angle"
print "           (m/s)      (m/s)     (deg ccw E)"  
print ' Model      {0:.2f}       {1:.2f}      {2:.0f}   '.format( 
         east_av[1], east_av[2], east_av[3]) 
print ' Nodes      {0:.2f}       {1:.2f}      {2:.0f}   '.format(
         maje, minore, ince)
#print ' CODAR      {0:.2f}       {1:.2f}      {2:.0f}   '.format(
#         maje_cod, minore_cod, ince_cod)


Central   Major-Axis   Minor-Axis     Angle
           (m/s)      (m/s)     (deg ccw E)
 Model      0.17       0.01      129   
 Nodes      0.18       0.05      109   
East   Major-Axis   Minor-Axis     Angle
           (m/s)      (m/s)     (deg ccw E)
 Model      0.31       -0.02      136   
 Nodes      0.26       -0.04      124   


The CODAR region only calculates the surface currents. These values are compared with the 0.5 m depth model outputs.

In [65]:
print "Central   Major-Axis   Minor-Axis     Angle"
print "           (m/s)      (m/s)     (deg ccw E)"  
print ' Model      {0:.2f}       {1:.2f}      {2:.0f}   '.format( 
         cen[0,1], cen[0,2], cen[0,3]) 
print ' CODAR      {0:.2f}       {1:.2f}      {2:.0f}   '.format(
         majc_cod, minorc_cod, incc_cod)


print "East   Major-Axis   Minor-Axis     Angle"
print "           (m/s)      (m/s)     (deg ccw E)"  
print ' Model      {0:.2f}       {1:.2f}      {2:.0f}   '.format( 
         east[0,1], east[0,2], east[0,3]) 
print ' CODAR      {0:.2f}       {1:.2f}      {2:.0f}   '.format(
         maje_cod, minore_cod, ince_cod)

Central   Major-Axis   Minor-Axis     Angle
           (m/s)      (m/s)     (deg ccw E)
 Model      0.20       -0.07      137   
 CODAR      0.11       0.08      127   
East   Major-Axis   Minor-Axis     Angle
           (m/s)      (m/s)     (deg ccw E)
 Model      0.39       -0.18      127   
 CODAR      0.14       -0.04      133   
